In [ ]:
import json
from glob import glob
from pprint import pprint
from tqdm import tqdm
import inflect

p = inflect.engine()

In [ ]:
with open("./ambiguous_questions.json") as f:
    ambiguous_questions = json.load(f)

with open("./ambiguous_questions_rebuilt.json") as f:
    ambiguous_questions_pr = json.load(f)

In [ ]:
with open("./sceneGraphs/train_sceneGraphs.json") as f:
    sceneGraphs = json.load(f)

In [ ]:
def get_same_name_entities(sceneGraph, target_object_id):
    
    entities = []
    
    target_object_name = sceneGraph['objects'][target_object_id]['name']
    for object_key, object_value in sceneGraph["objects"].items():
        if object_key != target_object_id and object_value['name'] == target_object_name:
            entities.append((object_key, object_value))
            
    return entities

In [ ]:
def make_unique_attribute_question(sceneGraph, target_object_id):
    
    questions = []
    
    target_object = sceneGraph["objects"][target_object_id]
    
    same_entities = get_same_name_entities(sceneGraph, target_object_id)
    
    target_attributes = set(target_object["attributes"])
    
    question_type = "attribute"
    
    if len(same_entities) == 0:
        return questions
    
    # print(target_attributes)
    
    for entity_key, entity_value in same_entities:
        target_attributes = target_attributes - set(entity_value['attributes'])
        
    yes_unique_attributes = target_attributes
    
    # print(target_attributes)
    
    be_verb = "Is" if p.singular_noun(target_object['name']) == False else "Are"
    
    for unique_attribute in yes_unique_attributes:
        
        intermediate_question = f"{be_verb} the {target_object['name']} {unique_attribute}?"
        intermediate_answer = "yes"

        questions.append({'question' : intermediate_question,
                          'answer' : intermediate_answer,
                          'type': question_type,
                          'label' : "positive"})
    
    intersection = None
    for entity_key, entity_value in same_entities:
        
        if intersection is None:
            intersection = set(entity_value['attributes'])
        else:
            intersection = intersection & set(entity_value['attributes'])
            
    if intersection is not None:
        no_unique_attributes = intersection - set(target_object["attributes"])

        # print(no_unique_attributes)
        
        for unique_attribute in no_unique_attributes:
            
            intermediate_question = f"{be_verb} the {target_object['name']} {unique_attribute}?"
            intermediate_answer = "no"

            questions.append({'question' : intermediate_question,
                            'answer' : intermediate_answer,
                            'type': question_type,
                            'label' : "positive"})
        
    return questions